# Dataset Generation Notebook
This notebook contains the code required to generate the challenge datasets from the raw sources. Checkout the accompanying paper for sources and links to said raw datasets.

In [1]:
#Import
import os
import pandas as pd
import numpy as np
from collections import defaultdict

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

____
## 1. Ingest and Parse Raw Data

#### Main Yield Data

In [2]:
#Read data
pth = 'data/Production_Crops_Livestock_E_All_Data/Production_Crops_Livestock_E_All_Data_NOFLAG.csv'
df1 = pd.read_csv(pth,encoding='latin-1')

In [3]:
df1.head(10)

,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Unit,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020
0,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,ha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5900.0,6000.0,6000.0,6000.0,5800.0,5800.0,5800.0,5700.0,5700.0,5600.0,5500.0,5500.0,5400.0,5400.0,6037.0,5500.0,5500.0,5500.0,5500.0,5500.0,5500.0,5500.0,5500.0,5500.0,7000.0,9000.0,5500.0,5700.0,12000.0,11768.0,12000.0,12000.0,12000.0,11029.0,11210.0,13469.0,13490.0,14114.0,13703.0,14676.0,19481.0,19793.0,20053.0,29203.0,22134.0
1,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5419,Yield,hg/ha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16610.0,15000.0,20000.0,17500.0,17069.0,13793.0,18966.0,17018.0,18421.0,16071.0,18182.0,16364.0,16667.0,16296.0,15736.0,16364.0,18000.0,16364.0,16364.0,16364.0,16364.0,16364.0,16364.0,20000.0,17143.0,16667.0,21407.0,24561.0,12250.0,13282.0,16667.0,26234.0,35000.0,39154.0,49955.0,45000.0,45960.0,29910.0,19996.0,16521.0,16859.0,13788.0,17161.0,13083.0,17759.0
2,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5510,Production,tonnes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,9800.0,9000.0,12000.0,10500.0,9900.0,8000.0,11000.0,9700.0,10500.0,9000.0,10000.0,9000.0,9000.0,8800.0,9500.0,9000.0,9900.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,11000.0,12000.0,15000.0,11774.0,14000.0,14700.0,15630.0,20000.0,31481.0,42000.0,43183.0,56000.0,60611.0,62000.0,42215.0,27400.0,24246.0,32843.0,27291.0,34413.0,38205.0,39307.0
3,2,'004,Afghanistan,711,'01654,"Anise, badian, coriander, cumin, caraway, fenn...",5312,Area harvested,ha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,700.0,700.0,300.0,1100.0,1300.0,1882.0,700.0,2270.0,2871.0,3000.0,7000.0,4000.0,12000.0,11311.0,4000.0,1600.0,3300.0,6800.0,15000.0,16000.0,17432.0,28000.0,15000.0,17748.0,17000.0,19500.0,18500.0,18500.0,30000.0,25000.0,24500.0,26500.0,25333.0,25444.0,25759.0
4,2,'004,Afghanistan,711,'01654,"Anise, badian, coriander, cumin, caraway, fenn...",5419,Yield,hg/ha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7143.0,7143.0,6667.0,7273.0,7692.0,7072.0,7143.0,6855.0,6761.0,6667.0,6000.0,6250.0,5917.0,6189.0,6250.0,6250.0,6061.0,6029.0,6000.0,6250.0,6222.0,6071.0,6000.0,6202.0,6000.0,6414.0,6757.0,6757.0,7167.0,7200.0,7075.0,7149.0,7142.0,7123.0,7138.0
5,2,'004,Afghanistan,711,'01654,"Anise, badian, coriander, cumin, caraway, fenn...",5510,Production,tonnes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500.0,500.0,200.0,800.0,1000.0,1331.0,500.0,1556.0,1941.0,2000.0,4200.0,2500.0,7100.0,7000.0,2500.0,1000.0,2000.0,4100.0,9000.0,10000.0,10846.0,17000.0,9000.0,11008.0,10200.0,12508.0,12500.0,12500.0,21500.0,18000.0,17333.0,18944.0,18093.0,18123.0,18387.0
6,2,'004,Afghanistan,515,'01341,Apples,5312,Area harvested,ha,2220.0,2220.0,2220.0,2350.0,2480.0,2740.0,3060.0,2990.0,3040.0,3060.0,3060.0,3150.0,3210.0,3270.0,3270.0,3270.0,3280.0,3250.0,3150.0,3150.0,3080.0,2960.0,2850.0,2730.0,2660.0,2590.0,2230.0,2330.0,2340.0,2365.0,2346.0,2308.0,2300.0,2350.0,2631.0,3135.0,3748.0,4410.0,4930.0,5598.0,5482.0,5045.0,3935.0,2311.0,2751.0,7000.0,7000.0,7000.0,8550.0,8550.0,8863.0,9148.0,10341.0,12247.0,13038.0,19365.0,26847.0,28381.0,27559.0,25643.0
7,2,'004,Afghanistan,515,'01341,Apples,5419,Yield,hg/ha,68018.0,68018.0,68018.0,78298.0,82258.0,83212.0,90196.0,93311.0,92763.0,89216.0,70261.0,84444.0,85981.0,85015.0,85321.0,80122.0,47256.0,69846.0,68571.0,70794.0,71753.0,72297.0,72982.0,73260.0,71805

In [4]:
#Selected
items_sl = ['Cereals',
            'Maize',
            'Sugar Crops Primary',
            'Potatoes',
            'Rice',
            'Rice, paddy (rice milled equivalent)',
            'Beans, dry',
            'Wheat',
            'Sorghum',
            'Barley',
            'Millet',
            'Oats',
            'Rye',
            'Buckwheat',
            'Molasses',
           ]

len(items_sl)

15

In [5]:
#Take stock of data we have on collected categories
df1 = df1.loc[df1.Item.isin(items_sl)]
df1.head(5)

,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Unit,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020
13,2,'004,Afghanistan,44,'0115,Barley,5312,Area harvested,ha,350000.0,350000.0,350000.0,350000.0,350000.0,350000.0,316000.0,317000.0,317000.0,315000.0,315000.0,320000.0,320000.0,340200.0,320000.0,310000.0,310000.0,310000.0,304000.0,306000.0,278000.0,257000.0,242000.0,234000.0,221000.0,213000.0,213000.0,212000.0,211000.0,205000.0,204000.0,250000.0,250000.0,250000.0,225000.0,220000.0,200000.0,200000.0,180000.0,124000.0,87000.0,236000.0,270000.0,315000.0,240000.0,236000.0,236000.0,236000.0,267000.0,212000.0,190000.0,280000.0,278000.0,342472.0,282000.0,219208.0,68179.0,84147.0,84070.0,86099.0
14,2,'004,Afghanistan,44,'0115,Barley,5419,Yield,hg/ha,10800.0,10800.0,10800.0,10857.0,10857.0,10714.0,11297.0,11388.0,11514.0,11746.0,11270.0,10938.0,11250.0,11170.0,12000.0,12903.0,9677.0,10484.0,10461.0,10490.0,10791.0,10739.0,10992.0,10855.0,10860.0,10845.0,10845.0,11085.0,10948.0,10537.0,10637.0,8000.0,10000.0,9200.0,8889.0,10000.0,12500.0,12000.0,12000.0,5968.0,10000.0,14619.0,8889.0,9206.0,14042.0,15424.0,15678.0,14110.0,18202.0,20613.0,16100.0,18000.0,18489.0,15213.0,14291.0,13770.0,13933.0,6748.0,14699.0,14838.0
15,2,'004,Afghanistan,44,'0115,Barley,5510,Production,tonnes,378000.0,378000.0,378000.0,380000.0,380000.0,375000.0,357000.0,361000.0,365000.0,370000.0,355000.0,350000.0,360000.0,380000.0,384000.0,400000.0,300000.0,325000.0,318000.0,321000.0,300000.0,276000.0,266000.0,254000.0,240000.0,231000.0,231000.0,235000.0,231000.0,216000.0,217000.0,200000.0,250000.0,230000.0,200000.0,220000.0,250000.0,240000.0,216000.0,74000.0,87000.0,345000.0,240000.0,290000.0,337000.0,364000.0,370000.0,333000.0,486000.0,437000.0,305900.0,504000.0,514000.0,521000.0,403000.0,301856.0,94995.0,56781.0,123576.0,127757.0
78,2,'004,Afghanistan,79,'0118,Millet,5312,Area harvested,ha,23500.0,24500.0,25500.0,26500.0,27500.0,28500.0,29500.0,29500.0,31500.0,33000.0,35000.0,36000.0,37000.0,38000.0,40000.0,42000.0,43000.0,42000.0,39500.0,39500.0,37000.0,35000.0,35000.0,32000.0,31000.0,29000.0,29000.0,29000.0,29000.0,30000.0,28071.0,25000.0,26882.0,27000.0,26982.0,26670.0,26401.0,26105.0,25871.0,27000.0,27000.0,25000.0,19700.0,19000.0,27000.0,12000.0,12000.0,12000.0,10000.0,5000.0,5000.0,5000.0,5000.0,1515.0,2693.0,3376.0,2864.0,2539.0,1401.0,1437.0
79,2,'004,Afghanistan,79,'0118,Millet,5419,Yield,hg/ha,8085.0,8163.0,8235.0,8302.0,8364.0,8421.0,8475.0,8814.0,8571.0,8485.0,8286.0,8333.0,8108.0,8158.0,8000.0,9524.0,8140.0,8571.0,8608.0,8608.0,8649.0,8571.0,8571.0,8750.0,8710.0,8621.0,8621.0,8621.0,8621.0,8667.0,8301.0,8000.0,8100.0,8148.0,8220.0,8217.0,8228.0,8238.0,8270.0,8148.0,8148.0,8000.0,8629.0,8947.0,8148.0,10000.0,16667.0,18333.0,19000.0,30000.0,30000.0,30000.0,26000.0,24845.0,23201.0,22269.0,21865.0,20729.0,19372.0,13187.0


In [6]:
#Count how many years of data are available and countries
country_ct = df1.Area.value_counts()
year_cols = df1.columns[9:]

non_na_entries = (~df1[year_cols].isna())
non_na_entries.sum().sum() #255030 datapoints

255030

#### Food Balance

In [7]:
#Food balance dataset
pth = 'data/FoodBalanceSheets_E_All_Data/FoodBalanceSheets_E_All_Data_NOFLAG.csv'
df_foodbal = pd.read_csv(pth,encoding='latin-1')

In [8]:
df_foodbal.head(20)

,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Unit,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019
0,2,'004,Afghanistan,2501,'S2501,Population,511,Total Population - Both sexes,1000 persons,29186.00,30117.00,31161.00,32270.00,33371.00,34414.00,35383.00,36296.00,37172.00,38042.00
1,2,'004,Afghanistan,2501,'S2501,Population,5301,Domestic supply quantity,1000 tonnes,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2,'004,Afghanistan,2901,'S2901,Grand Total,664,Food supply (kcal/capita/day),kcal/capita/day,2170.00,2152.00,2159.00,2196.00,2265.00,2250.00,2228.00,2303.00,2270.00,2273.00
3,2,'004,Afghanistan,2901,'S2901,Grand Total,674,Protein supply quantity (g/capita/day),g/capita/day,59.23,58.00,57.82,57.71,60.17,58.45,58.46,59.50,57.62,57.31
4,2,'004,Afghanistan,2901,'S2901,Grand Total,684,Fat supply quantity (g/capita/day),g/capita/day,36.69,34.81,36.53,37.10,41.48,38.28,40.70,40.03,41.46,39.78
5,2,'004,Afghanistan,2903,'S2903,Vegetal Products,664,Food supply (kcal/capita/day),kcal/capita/day,1964.00,1953.00,1955.00,1993.00,2019.00,2038.00,2024.00,2108.00,2081.00,2087.00
6,2,'004,Afghanistan,2903,'S2903,Vegetal Products,674,Protein supply quantity (g/capita/day),g/capita/day,47.42,46.44,45.92,46.02,46.21,46.37,46.93,48.34,46.94,46.97
7,2,'004,Afghanistan,2903,'S2903,Vegetal Products,684,Fat supply quantity (g/capita/day),g/capita/day,22.33,20.93,22.56,22.93,24.80,23.82,26.66,26.82,28.61,27.11
8,2,'004,Afghanistan,2941,'S2941,Animal Products,664,Food supply (kcal/capita/day),kcal/capita/day,206.00,200.00,204.00,204.00,246.00,211.00,204.00,195.00,189.00,187.00
9,2,'004,Afghanistan,2941,'S2941,Animal Products,674,Protein supply quantity (g/capita/day),g/capita/day,11.81,11.57,11.90,11.69,13.96,12.08,11.54,11.16,10.69,10.34


In [9]:
df_foodbal.Item.value_counts()

Milk - Excluding Butter            6813
Eggs                               6606
Miscellaneous (GIFT)               5791
Animal fats                        3639
Cereals - Excluding Beer           3635
Starchy Roots                      3634
Sugar & Sweeteners                 3633
Oilcrops                           3633
Stimulants                         3633
Pulses                             3633
Fruits - Excluding Wine            3632
Vegetables                         3632
Meat                               3630
Treenuts                           3629
Offals                             3626
Spices                             3626
Vegetable Oils                     3596
Alcoholic Beverages                3507
Maize and products                 3346
Sugar Crops                        3318
Wheat and products                 3305
Potatoes and products              3287
Rice and products                  3264
Barley and products                3109
Oilcrops, Other                    3069


In [10]:
df_foodbal.Element.value_counts()

#Use Food, Feed, Losses
#For info of def check out https://www.fao.org/faostat/en/#data/FBS

Domestic supply quantity                  25113
Import Quantity                           24640
Protein supply quantity (g/capita/day)    23857
Fat supply quantity (g/capita/day)        23852
Food supply (kcal/capita/day)             23833
Food                                      23402
Food supply quantity (kg/capita/yr)       23305
Export Quantity                           23043
Residuals                                 22126
Stock Variation                           21214
Production                                19162
Losses                                    12772
Other uses (non-food)                     12714
Processing                                11326
Feed                                      11034
Tourist consumption                       10911
Seed                                       7154
Total Population - Both sexes               214
Name: Element, dtype: int64

In [11]:
df2 = df_foodbal.loc[df_foodbal.Element.isin(['Food','Feed','Losses'])]
df2.head(5) #include a lot of other non selected item elements, to filter or not to filter?

,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Unit,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019
16,2,'004,Afghanistan,2905,'S2905,Cereals - Excluding Beer,5521,Feed,1000 tonnes,789.0,679.0,765.0,896.0,704.0,748.0,472.0,403.0,410.0,457.0
18,2,'004,Afghanistan,2905,'S2905,Cereals - Excluding Beer,5123,Losses,1000 tonnes,837.0,647.0,911.0,936.0,963.0,837.0,800.0,706.0,595.0,809.0
23,2,'004,Afghanistan,2905,'S2905,Cereals - Excluding Beer,5142,Food,1000 tonnes,5638.0,5690.0,5644.0,5937.0,6072.0,6265.0,6426.0,6746.0,6761.0,6907.0
33,2,'004,Afghanistan,2511,'S2511,Wheat and products,5521,Feed,1000 tonnes,294.0,282.0,188.0,326.0,141.0,260.0,54.0,221.0,276.0,249.0
35,2,'004,Afghanistan,2511,'S2511,Wheat and products,5123,Losses,1000 tonnes,678.0,508.0,758.0,775.0,803.0,701.0,684.0,642.0,542.0,734.0


In [12]:
df2['Item'].unique() #just take all for now

array(['Cereals - Excluding Beer', 'Wheat and products',
       'Rice and products', 'Barley and products', 'Maize and products',
       'Rye and products', 'Oats', 'Millet and products',
       'Sorghum and products', 'Cereals, Other', 'Starchy Roots',
       'Cassava and products', 'Potatoes and products', 'Sweet potatoes',
       'Roots, Other', 'Sugar Crops', 'Sugar cane', 'Sugar beet',
       'Sugar & Sweeteners', 'Sugar (Raw Equivalent)',
       'Sweeteners, Other', 'Honey', 'Pulses', 'Beans', 'Peas',
       'Pulses, Other and products', 'Treenuts', 'Nuts and products',
       'Oilcrops', 'Soyabeans', 'Groundnuts', 'Sunflower seed',
       'Rape and Mustardseed', 'Cottonseed', 'Coconuts - Incl Copra',
       'Sesame seed', 'Palm kernels', 'Olives (including preserved)',
       'Oilcrops, Other', 'Vegetable Oils', 'Soyabean Oil',
       'Groundnut Oil', 'Sunflowerseed Oil', 'Rape and Mustard Oil',
       'Cottonseed Oil', 'Palmkernel Oil', 'Palm Oil', 'Coconut Oil',
       'Sesame

In [13]:
#Aggregate
df2_feed = df2.loc[df2.Element == 'Feed'].groupby(['Area Code']).sum()
df2_feed.drop(['Item Code','Element Code'],axis=1,inplace=True)
df2_feed['units'] = '1000 tonnes'

In [14]:
#Aggregate
df2_losses = df2.loc[df2.Element == 'Losses'].groupby(['Area Code']).sum()
df2_losses.drop(['Item Code','Element Code'],axis=1,inplace=True)
df2_losses['units'] = '1000 tonnes'

In [15]:
#Aggregate
df2_Food = df2.loc[df2.Element == 'Food'].groupby(['Area Code']).sum()
df2_Food.drop(['Item Code','Element Code'],axis=1,inplace=True)
df2_Food['units'] = '1000 tonnes'

#### Produce Prices (SKIP)

In [21]:
#Produce prices
pth = 'data/Prices_E_All_Data/Prices_E_All_Data_NOFLAG.csv'
df_prod_price = pd.read_csv(pth,encoding='latin-1')


In [23]:
df_prod_price.Item.value_counts() #just agg all for now

Tomatoes                                                                                                         1710
Hen eggs in shell, fresh                                                                                         1701
Potatoes                                                                                                         1659
Maize (corn)                                                                                                     1637
Raw milk of cattle                                                                                               1608
Cabbages                                                                                                         1507
Cucumbers and gherkins                                                                                           1424
Meat of chickens, fresh or chilled (biological)                                                                  1379
Meat of cattle with the bone, fresh or chilled (biologic

In [24]:
#Elements of interest
df_prod_price.Element.value_counts() #not very useful to be in LCU since that will need conversion to USD

Producer Price (LCU/tonne)                58084
Producer Price Index (2014-2016 = 100)    13194
Producer Price (SLC/tonne)                 8505
Producer Price (USD/tonne)                 8306
Name: Element, dtype: int64

In [25]:
df_prod_price.loc[df_prod_price.Element=='Producer Price (USD/tonne)'] 

#SKIP due to lack of USD data

,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Months Code,Months,Unit,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021
125,3,'008,Albania,515,'01341,Apples,5532,Producer Price (USD/tonne),7021,Annual value,USD,NaN,NaN,461.0,NaN,323.6,325.4,253.1,248.9,270.2,257.5,257.9,274.7,317.6,375.6,390.5,428.1,486.6,476.8,463.3,356.0,386.5,369.7,492.1,474.8,389.0,381.3,461.6,435.2,346.7,510.3,NaN
135,3,'008,Albania,526,'01343,Apricots,5532,Producer Price (USD/tonne),7021,Annual value,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,891.2,530.1,486.6,452.9,526.4,452.2,584.8,489.9,823.3,758.4,746.3,732.1,599.1,777.8,909.8,724.3,NaN
151,3,'008,Albania,44,'0115,Barley,5532,Producer Price (USD/tonne),7021,Annual value,USD,NaN,NaN,NaN,NaN,NaN,287.1,248.4,325.3,305.0,313.1,299.7,306.8,352.9,401.8,400.5,305.8,337.3,464.9,410.6,307.9,339.0,316.1,482.6,431.4,341.4,319.7,303.6,302.2,306.1,286.6,NaN
167,3,'008,Albania,176,'01701,"Beans, dry",5532,Producer Price (USD/tonne),7021,Annual value,USD,NaN,NaN,447.0,NaN,873.8,602.9,725.2,956.0,806.2,528.8,808.4,770.6,593.3,1053.7,1173.5,1181.4,1017.4,1609.2,1642.5,1356.6,1328.1,1562.1,2403.7,2059.6,1738.6,1661.7,1710.4,2078.7,1978.8,2100.9,NaN
184,3,'008,Albania,420,'01243,"Broad beans and horse beans, green",5532,Producer Price (USD/tonne),7021,Annual value,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1447.9,907.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88021,181,'716,Zimbabwe,156,'01802,Sugar cane,5532,Producer Price (USD/tonne),7021,Annual value,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,536.0,550.0,574.0,NaN,NaN,NaN,NaN
88025,181,'716,Zimbabwe,122,'01530,Sweet potatoes,5532,Producer Price (USD/tonne),7021,Annual value,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,325.0,350.0,330.0,NaN,NaN,NaN,NaN
88043,181,'716,Zimbabwe,388,'01234,Tomatoes,5532,Producer Price (USD/tonne),7021,Annual value,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,504.0,567.0,786.0,NaN,NaN,NaN,NaN
88059,181,'716,Zimbabwe,826,'01970,Unmanufactured tobacco,5532,Producer Price (USD/tonne),7021,Annual value,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2890.0,2730.1,3670.0,NaN,3160.0,2500.0,2950.0,2960.0,2920.0,NaN,NaN,NaN


#### Macro Data

In [17]:
#Try Macro data
pth = 'data/Macro-Statistics_Key_Indicators_E_All_Data/Macro-Statistics_Key_Indicators_E_All_Data_NOFLAG.csv'
df_macro = pd.read_csv(pth,encoding='latin-1')


In [18]:
df_macro.head(5)

,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Unit,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020
0,2,'004,Afghanistan,22008,Gross Domestic Product,6109,Value Local Currency,millions,77.911878,81.574837,74.152916,76.619555,92.774575,104.385259,113.107176,128.917758,145.028303,158.776212,160.930107,169.591475,173.589299,172.453817,170.475688,168.520249,168.035667,150.976242,137.322243,161.491528,180.109889,172.172997,174.301038,143.927675,729.118301,2297.365030,6674.524156,8773.855716,8805.572314,100567.527295,158270.632870,170927.404578,195740.846106,230601.118468,257813.610332,307884.813631,354705.069349,470250.593341,514400.361283,583525.091849,682799.599547,811093.070883,9.744573e+05,1.086606e+06,1.119229e+06,1.144181e+06,1.222917e+06,1.285460e+06,1.327690e+06,1.469596e+06,1.520390e+06
1,2,'004,Afghanistan,22008,Gross Domestic Product,6110,Value US$,millions,1731.435587,1812.837521,1647.900178,1702.716294,2061.729287,2319.753506,2513.580659,2864.939207,3222.963833,3630.767856,3646.796868,3427.482014,3430.645143,3408.204613,3369.110951,3330.465681,3320.888887,2983.743471,2713.899488,3191.557068,3559.511736,3402.654940,3444.711414,2844.442641,1715.169169,2756.838036,2669.785634,2924.589326,2935.139404,2157.197316,3342.034168,3598.470576,4141.523943,4729.042179,5388.482107,6220.574147,7104.711445,9412.161765,10236.901594,11595.133469,14698.889678,17350.694945,1.913650e+04,1.962180e+04,1.955070e+04,1.871305e+04,1.801956e+04,1.889635e+04,1.841885e+04,1.890449e+04,1.979326e+04
2,2,'004,Afghanistan,22008,Gross Domestic Product,6119,Value US$ per capita,US$,154.957138,157.975357,139.756605,140.616194,166.095029,182.813796,194.202472,217.513678,241.579792,270.729453,273.035141,260.216148,266.302169,271.835860,276.059517,278.975344,282.961677,257.118767,233.594278,268.901352,286.772753,255.857628,237.803353,179.839036,100.444869,152.221868,141.607370,151.085927,148.706776,106.946309,160.829727,166.541981,183.247028,199.698828,217.921744,242.477079,268.781382,347.305373,369.266275,408.353930,503.636606,576.101770,6.141096e+02,6.080586e+02,5.858627e+02,5.437689e+02,5.092712e+02,5.206164e+02,4.955043e+02,4.969406e+02,5.084537e+02
3,2,'004,Afghanistan,22008,Gross Domestic Product,6183,"Value Local Currency, 2015 prices",millions,640024.694809,638260.085798,556498.281260,545260.592016,571963.541311,606538.115012,640652.358105,622700.035211,650881.544583,647761.511702,625359.636217,630423.153215,643414.185241,670981.813064,687706.446508,691900.378533,708420.513683,656098.152210,598661.464479,554676.102558,532223.150831,483512.600665,476587.229752,385407.968239,296737.925985,387337.309453,401148.285358,380405.169744,362609.298241,345273.395382,326789.905499,308373.050408,458140.931411,530558.854216,545734.797948,586710.864491,636952.631554,781177.101080,772544.956581,856508.039841,901472.494035,962434.513050,1.055821e+06,1.126041e+06,1.160649e+06,1.144181e+06,1.161900e+06,1.192655e+06,1.206839e+06,1.254046e+06,1.229783e+06
4,2,'004,Afghanistan,22008,Gross Domestic Product,6185,"Value US$ per capita, 2015 prices",US$,937.000000,910.000000,772.000000,736.000000,754.000000,782.000000,810.000000,773.000000,798.000000,790.000000,766.000000,783.000000,817.000000,875.000000,922.000000,948.000000,987.000000,925.000000,843.000000,764.000000,701.000000,595.000000,538.000000,399.000000,284.000000,350.000000,348.000000,321.000000,300.000000,280.000000,257.000000,233.000000,332.000000,366.000000,361.000000,374.000000,394.000000,471.000000,456.000000,493.000000,505.000000,523.000000,5.540000e+02,5.710000e+02,5.690000e+02,5.440000e+02,5.370000e+02,5.370000e+02,5.310000e+02,5.390000e+02,5.170000e+02


In [19]:
df_macro.loc[df_macro.Element=='Value US$'].Unit.unique()

array(['millions'], dtype=object)

In [20]:
df_macro.Item.value_counts()

#select
macro_it_se = ['Gross Domestic Product',
               'Gross Output (Agriculture)']

In [21]:
df_macro.Element.value_counts()

#select
macro_el_se = ['Value US$']

In [22]:
df3_gdp = df_macro.loc[(df_macro.Item.isin(['Gross Domestic Product'])) & (df_macro.Element.isin(macro_el_se))]
df3_agi_out = df_macro.loc[(df_macro.Item.isin(['Gross Output (Agriculture)'])) & (df_macro.Element.isin(macro_el_se))]

In [23]:
#Aggregate
df3_gdp = df3_gdp.groupby(['Area Code']).sum() #can skip Element agg here, just 1 value
df3_gdp.drop(['Item Code','Element Code'],axis=1,inplace=True)
df3_gdp['units'] = 'millions $'

In [24]:
df3_agi_out = df3_agi_out.groupby(['Area Code']).sum() #can skip Element agg here, just 1 value
df3_agi_out.drop(['Item Code','Element Code'],axis=1,inplace=True)
df3_agi_out['units'] = 'millions $'

In [27]:
df3_gdp.head(5)

,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,units
Area Code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2301.720664,2101.656351,1251.976343,1168.728174,1257.275245,1372.097033,1703.037605,1747.514993,2017.852886,1968.057835,2038.534803,2259.108208,2534.163311,2993.550393,3814.176694,5225.971850,6808.524718,9817.806812,12436.664185,9222.356746,9875.377779,10815.777611,10619.320693,11121.464430,11609.513241,10553.337528,10546.136240,11527.458713,12457.940705,13619.290546,12641.210873,millions $
2,1731.435587,1812.837521,1647.900178,1702.716294,2061.729287,2319.753506,2513.580659,2864.939207,3222.963833,3630.767856,3646.796868,3427.482014,3430.645143,3408.204613,3369.110951,3330.465681,3320.888887,2983.743471,2713.899488,3191.557068,3559.511736,3402.654940,3444.711414,2844.442641,1715.169169,2756.838036,2669.785634,2924.589326,2935.139404,2157.197316,3342.034168,3598.470576,4141.523943,4729.042179,5388.482107,6220.574147,7104.711445,9412.161765,10236.901594,11595.133469,14698.889678,17350.694945,19136.499341,19621.802455,19550.702567,18713.048410,18019.558182,18896.352021,18418.848300,18904.490263,19793.262413,millions $
3,2265.577056,2330.584860,2397.526448,2466.543461,2537.015817,2610.110369,2685.745678,2760.746397,2842.275926,2372.305641,2141.837051,2215.474766,2280.424885,2305.384485,2275.736342,2324.414456,2397.086883,2378.194810,2460.399974,2940.586487,2145.751209,1597.139589,1516.132824,1700.439125,1880.950821,2392.764887,3199.642024,2224.654241,2554.868837,3221.670165,3487.586302,3926.887597,4355.865889,5561.459461,7177.030365,8052.079314,8896.076004,10677.322220,12881.352058,12044.210390,11926.948284,12890.873323,12319.779604,12776.280738,13228.240079,11386.924853,11861.356451,13019.729856,15156.439968,15400.237155,14909.619335,millions $
4,5155.120658,5363.676160,7176.428384,9229.622200,13259.764650,15555.547083,17750.032488,20990.939140,26373.761387,33201.433009,42252.197741,44264.573122,45094.842770,48702.201863,52827.902259,57866.207279,62922.803868,64333.578322,58655.419182,55344.476937,61751.375957,46565.003199,49105.553595,50846.930343,42330.711146,41971.488420,46836.290499,48068.457028,48079.007974,48531.031757,54666.896857,55056.733057,56819.210891,67863.851626,85332.517817,103198.216067,117027.277939,134977.088396,171000.692135,137211.008421,161207.306305,200019.103013,209059.083118,209755.003251,213809.979480,165979.118791,160034.211700,170096.984537,175415.180411,171158.234285,147688.693830,millions $
6,99.476787,113.128492,143.506004,190.863792,236.055468,278.514317,287.588366,321.390137,389.745929,520.742333,564.812657,492.189978,475.654368,414.836776,417.649689,438.742504,609.881940,773.493497,912.838474,1006.552718,1302.006184,1400.576915,1531.034035,1274.297409,1287.524766,1491.495988,1548.787643,1493.901088,1533.515782,1568.801032,1429.044662,1546.913267,1755.992896,2361.635625,2895.042252,3159.831266,3456.261316,3952.395237,4085.783663,3674.186277,3446.565601,3624.386229,3188.830334,3192.603086,3267.334446,2788.757749,2894.646184,3000.160225,3218.419456,3155.149348,2864.399402,millions $


#### Inputs_Pesticides_Use_E_All_Data

In [29]:
#Try Macro data
pth = 'data/Inputs_Pesticides_Use_E_All_Data/Inputs_Pesticides_Use_E_All_Data_NOFLAG.csv'
df_pest = pd.read_csv(pth,encoding='latin-1')


In [30]:
df_pest.head(5)

,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Unit,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020
0,3,'008,Albania,1357,Pesticides (total),5157,Agricultural Use,tonnes,121.0,121.0,121.0,121.0,201.0,251.0,262.40,273.79,285.19,296.59,307.98,319.38,330.78,342.17,353.57,364.97,376.36,387.76,399.16,410.55,590.5,582.68,361.62,450.60,457.47,538.24,584.49,614.91,442.35,729.67,756.51
1,3,'008,Albania,1309,Insecticides,5157,Agricultural Use,tonnes,70.0,70.0,70.0,70.0,117.0,145.0,149.92,154.84,159.76,164.68,169.60,174.52,179.44,184.36,189.28,194.20,199.12,204.04,208.96,213.88,375.8,317.07,157.57,214.13,34.48,148.69,182.28,191.37,119.32,141.79,146.75
2,3,'008,Albania,1310,Insecticides  Chlorinated Hydrocarbons,5157,Agricultural Use,tonnes,NaN,NaN,NaN,10.0,17.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,'008,Albania,1311,Insecticides  Organo-phosphates,5157,Agricultural Use,tonnes,NaN,NaN,NaN,30.0,50.0,64.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,'008,Albania,1312,Insecticides  Carbamates,5157,Agricultural Use,tonnes,NaN,NaN,NaN,14.0,26.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df_pest.Element.value_counts()

Agricultural Use    4767
Name: Element, dtype: int64

In [38]:
df_pest.Item.value_counts() #just use Pesticides (Total)

Pesticides (total)                                      207
Insecticides                                            207
Fungicides and Bactericides                             205
Herbicides                                              201
Rodenticides                                            171
Plant Growth Regulators                                 149
Insecticides  Organo-phosphates                        141
Insecticides  Carbamates                               139
Insecticides  Pyrethroids                              137
Herbicides  Other                                      133
Insecticides  Other                                    133
Other Pesticides nes                                    131
Fung & Bact  Dithiocarbamates                          131
Fung & Bact  Other                                     131
Fung & Bact  Benzimidazoles                            124
Fung & Bact  Inorganics                                122
Herbicides  Phenoxy hormone products   

In [32]:
df4 = df_pest.loc[(df_pest.Item.isin(['Pesticides (total)']))]

In [33]:
df4.shape

(207, 39)

In [34]:
#Aggregate
df4 = df4.groupby(['Area Code']).sum() #can skip Element agg here, just 1 value
df4.drop(['Item Code','Element Code'],axis=1,inplace=True)
df4['units'] = 'tonnes'

In [35]:
df4.head(5)

,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,units
Area Code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.00,0.00,8.00,8.00,23.00,23.90,32.00,41.46,50.92,60.38,69.85,79.31,88.77,98.23,204.77,220.51,227.55,241.71,218.20,198.36,278.72,320.67,362.62,404.57,446.52,488.47,530.42,572.37,614.32,527.33,639.15,tonnes
3,121.00,121.00,121.00,121.00,201.00,251.00,262.40,273.79,285.19,296.59,307.98,319.38,330.78,342.17,353.57,364.97,376.36,387.76,399.16,410.55,590.50,582.68,361.62,450.60,457.47,538.24,584.49,614.91,442.35,729.67,756.51,tonnes
4,5743.46,7094.39,3876.18,3710.00,4058.62,6255.63,2190.88,1822.16,1498.96,2228.17,2537.40,2846.64,3155.87,3465.11,3774.35,2810.61,2504.01,3562.52,5720.11,3168.41,1869.58,7612.21,8980.17,8269.57,7151.52,4516.93,4153.38,6067.35,6067.35,6067.35,6067.35,tonnes
7,64.00,79.00,23.00,169.00,25.50,336.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,tonnes
8,7.52,7.52,7.52,7.52,7.52,7.52,7.52,7.52,7.52,7.52,7.52,7.52,7.52,7.52,7.52,7.52,7.52,7.52,7.52,7.52,7.52,39.26,31.50,28.30,27.47,26.64,26.86,27.07,27.28,27.49,13.98,tonnes


#### Fire

In [36]:
pth = 'data/Emissions_Land_Use_Fires_E_All_Data/Emissions_Land_Use_Fires_E_All_Data_NOFLAG.csv'
df_fire = pd.read_csv(pth,encoding='latin-1')


In [39]:
df5 = df_fire.loc[df_fire.Element == 'Burned Area']

In [40]:
#Aggregate
df5 = df5.groupby(['Area Code']).sum() #can skip Element agg here, just 1 value
df5.drop(['Item Code','Element Code'],axis=1,inplace=True)
df5['units'] = 'ha'

In [41]:
df5.head(5)

,Source Code,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021,units
Area Code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,36600,0.0000,0.0000,17669.3778,17669.3778,17669.3778,17669.3778,5735.4770,4721.5916,8645.8978,4556.3209,7314.2173,14847.4239,1289.2230,1289.4888,22324.2809,17838.5041,35327.8898,1461.2208,17989.8205,214.8818,132381.7684,7049.3520,7628.3088,22562.0843,6726.2355,16761.9186,14718.4514,31438.2078,19940.4534,28171.6628,5221.0029,15319.3134,ha
2,36600,41789.0604,41789.0604,41789.0604,41789.0604,41789.0604,41789.0604,72895.2688,54694.9448,133414.4631,86039.9461,39746.9974,944.6109,36798.6118,225988.0717,48480.2702,21253.7342,14096.7402,9336.3286,110282.1315,9272.6787,99674.3386,7983.9494,11765.5912,12838.2306,5000.8780,3435.0318,6845.7262,1674.5291,816.0738,28102.1642,33890.4075,11243.8526,ha
3,36600,28419.3604,28419.3604,28419.3604,28419.3604,28419.3604,28419.3604,7601.2915,16029.5436,11165.3082,19293.4012,33332.8808,8122.7657,0.0000,24801.3069,966.9855,10894.3425,2794.0077,250578.6492,19769.9899,8123.2016,10830.0422,65164.9968,106775.0010,1504.4410,580.2590,11801.5814,4621.1300,53133.1387,8061.3952,6362.6964,12402.5098,44192.5049,ha
4,36600,197060.3271,197060.3271,197060.3271,197060.3271,197060.3271,197060.3271,163698.5796,127795.8250,304272.4936,180927.0461,400339.7605,113233.1715,76669.8648,178887.4274,155938.1133,196184.9818,99647.0161,399268.4821,195521.2010,260918.2108,147159.4068,130506.5714,575251.6549,50517.0514,252858.5246,109471.4144,89032.9895,262165.1968,2769.7249,113857.0598,215463.4848,321213.2513,ha
5,36600,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,ha


#### abcnews-date-text


In [43]:
pth = 'data/abcnews-date-text.csv'
df_news = pd.read_csv(pth)


In [44]:
print(df_news.shape)

df_news.head(10)

(1244184, 2)


,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers
5,20030219,ambitious olsson wins triple jump
6,20030219,antic delighted with record breaking barca
7,20030219,aussie qualifier stosur wastes four memphis match
8,20030219,aust addresses un security council over iraq
9,20030219,australia is locked into war timetable opp


In [45]:
farms = []

for i in df_news.headline_text.values:
    if 'farm' in i or 'agri' in i:
        farms.append(i)

In [46]:
len(farms)

17308

In [50]:
print(farms[:10])

['plan to encourage farmers into plantation timber', 'groups praise outgoing opposition agriculture', 'rain boosts farmers confidence', 'council approves poultry farm', 'farmers celebrate good rain', 'group gets clearer understanding of farmers gm', 'vic govt gets farmers drought update', 'victorian farmers hope rain will break drought', 'rain brings relief for act farmers', 'drought assistance will continue for farmers carr']


#### NCEI Weather (SKIP)

In [72]:
pth = 'data/ncei_weather_data/gsom-latest'

In [331]:
df_w1 = pd.read_csv(pth + '/US10siou001.csv')


___
## 2. Merge Datasets

In [47]:
df1.index = df1['Area Code']
df1.drop(['Area Code','Area Code (M49)'],axis=1,inplace=True)

In [48]:
#Fine tune df1
df1.Element.value_counts() #Let's concentrate on production?

df1_ = df1.loc[df1.Element == 'Production']

In [49]:
df1_.head(1)

,Area,Item Code,Item Code (CPC),Item,Element Code,Element,Unit,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020
Area Code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,Afghanistan,44,'0115,Barley,5510,Production,tonnes,378000.0,378000.0,378000.0,380000.0,380000.0,375000.0,357000.0,361000.0,365000.0,370000.0,355000.0,350000.0,360000.0,380000.0,384000.0,400000.0,300000.0,325000.0,318000.0,321000.0,300000.0,276000.0,266000.0,254000.0,240000.0,231000.0,231000.0,235000.0,231000.0,216000.0,217000.0,200000.0,250000.0,230000.0,200000.0,220000.0,250000.0,240000.0,216000.0,74000.0,87000.0,345000.0,240000.0,290000.0,337000.0,364000.0,370000.0,333000.0,486000.0,437000.0,305900.0,504000.0,514000.0,521000.0,403000.0,301856.0,94995.0,56781.0,123576.0,127757.0


In [50]:
#Merge! 
col1 = [i + '_AgriProd' if 'Y' in i else i for i in df1_.columns ]
col2 = [i + '_AnnualFeedProd' for i in df2_feed.columns]

df1_.columns = col1
df2_feed.columns = col2 

In [51]:
df = df1_.merge(df2_feed,how='left',right_index=True,left_index=True)

In [52]:
#Merge! 
col2 = [i + '_AnnualFoodProd' for i in df2_Food.columns]

df2_Food.columns = col2 

In [53]:
df = df.merge(df2_Food,how='left',right_index=True,left_index=True)

In [54]:
#Merge! 
col2 = [i + '_AnnualAgriLosses' for i in df2_losses.columns]

df2_losses.columns = col2 

In [55]:
df = df.merge(df2_losses,how='left',right_index=True,left_index=True)

In [56]:
#Merge! 
col2 = [i + '_GDP' for i in df3_gdp.columns]

df3_gdp.columns = col2 

In [57]:
df = df.merge(df3_gdp,how='left',right_index=True,left_index=True)

In [58]:
#Merge! 
col2 = [i + '_GrossAgriVal' for i in df3_agi_out.columns]

df3_agi_out.columns = col2 

In [59]:
df = df.merge(df3_agi_out,how='left',right_index=True,left_index=True)

In [60]:
#Merge! 
col2 = [i + '_Pesticide' for i in df4.columns]

df4.columns = col2 

In [61]:
df = df.merge(df4,how='left',right_index=True,left_index=True)

In [62]:
#Merge! 
df5.drop('Source Code',axis=1,inplace=True)

col2 = [i + '_Fire' for i in df5.columns]

df5.columns = col2 

In [63]:
df = df.merge(df5,how='left',right_index=True,left_index=True)

In [64]:
df.shape

(2005, 269)

##### Done Merging!

In [65]:
df_news #this is unmerged, leave to the sideas a separate dataset and handled in feature engineering

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers
...,...,...
1244179,20211231,two aged care residents die as state records 2...
1244180,20211231,victoria records 5;919 new cases and seven deaths
1244181,20211231,wa delays adopting new close contact definition
1244182,20211231,western ringtail possums found badly dehydrate...


In [68]:
df.shape

(2005, 269)

In [69]:
df

,Area,Item Code,Item Code (CPC),Item,Element Code,Element,Unit,Y1961_AgriProd,Y1962_AgriProd,Y1963_AgriProd,Y1964_AgriProd,Y1965_AgriProd,Y1966_AgriProd,Y1967_AgriProd,Y1968_AgriProd,Y1969_AgriProd,Y1970_AgriProd,Y1971_AgriProd,Y1972_AgriProd,Y1973_AgriProd,Y1974_AgriProd,Y1975_AgriProd,Y1976_AgriProd,Y1977_AgriProd,Y1978_AgriProd,Y1979_AgriProd,Y1980_AgriProd,Y1981_AgriProd,Y1982_AgriProd,Y1983_AgriProd,Y1984_AgriProd,Y1985_AgriProd,Y1986_AgriProd,Y1987_AgriProd,Y1988_AgriProd,Y1989_AgriProd,Y1990_AgriProd,Y1991_AgriProd,Y1992_AgriProd,Y1993_AgriProd,Y1994_AgriProd,Y1995_AgriProd,Y1996_AgriProd,Y1997_AgriProd,Y1998_AgriProd,Y1999_AgriProd,Y2000_AgriProd,Y2001_AgriProd,Y2002_AgriProd,Y2003_AgriProd,Y2004_AgriProd,Y2005_AgriProd,Y2006_AgriProd,Y2007_AgriProd,Y2008_AgriProd,Y2009_AgriProd,Y2010_AgriProd,Y2011_AgriProd,Y2012_AgriProd,Y2013_AgriProd,Y2014_AgriProd,Y2015_AgriProd,Y2016_AgriProd,Y2017_AgriProd,Y2018_AgriProd,Y2019_AgriProd,Y2020_AgriProd,Y2010_AnnualFeedProd,Y2011_AnnualFeedProd,Y2012_AnnualFeedProd,Y2013_AnnualFeedProd,Y2014_AnnualFeedProd,Y2015_AnnualFeedProd,Y2016_AnnualFeedProd,Y2017_AnnualFeedProd,Y2018_AnnualFeedProd,Y2019_AnnualFeedProd,units_AnnualFeedProd,Y2010_AnnualFoodProd,Y2011_AnnualFoodProd,Y2012_AnnualFoodProd,Y2013_AnnualFoodProd,Y2014_AnnualFoodProd,Y2015_AnnualFoodProd,Y2016_AnnualFoodProd,Y2017_AnnualFoodProd,Y2018_AnnualFoodProd,Y2019_AnnualFoodProd,units_AnnualFoodProd,Y2010_AnnualAgriLosses,Y2011_AnnualAgriLosses,Y2012_AnnualAgriLosses,Y2013_AnnualAgriLosses,Y2014_AnnualAgriLosses,Y2015_AnnualAgriLosses,Y2016_AnnualAgriLosses,Y2017_AnnualAgriLosses,Y2018_AnnualAgriLosses,Y2019_AnnualAgriLosses,units_AnnualAgriLosses,Y1970_GDP,Y1971_GDP,Y1972_GDP,Y1973_GDP,Y1974_GDP,Y1975_GDP,Y1976_GDP,Y1977_GDP,Y1978_GDP,Y1979_GDP,Y1980_GDP,Y1981_GDP,Y1982_GDP,Y1983_GDP,Y1984_GDP,Y1985_GDP,Y1986_GDP,Y1987_GDP,Y1988_GDP,Y1989_GDP,Y1990_GDP,Y1991_GDP,Y1992_GDP,Y1993_GDP,Y1994_GDP,Y1995_GDP,Y1996_GDP,Y1997_GDP,Y1998_GDP,Y1999_GDP,Y2000_GDP,Y2001_GDP,Y2002_GDP,Y2003_GDP,Y2004_GDP,Y2005_GDP,Y2006_GDP,Y2007_GDP,Y2008_GDP,Y2009_GDP,Y2010_GDP,Y2011_GDP,Y2012_GDP,Y2013_GDP,Y2014_GDP,Y2015_GDP,Y2016_GDP,Y2017_GDP,Y2018_GDP,Y2019_GDP,Y2020_GDP,units_GDP,Y1970_GrossAgriVal,Y1971_GrossAgriVal,Y1972_GrossAgriVal,Y1973_GrossAgriVal,Y1974_GrossAgriVal,Y1975_GrossAgriVal,Y1976_GrossAgriVal,Y1977_GrossAgriVal,Y1978_GrossAgriVal,Y1979_GrossAgriVal,Y1980_GrossAgriVal,Y1981_GrossAgriVal,Y1982_GrossAgriVal,Y1983_GrossAgriVal,Y1984_GrossAgriVal,Y1985_GrossAgriVal,Y1986_GrossAgriVal,Y1987_GrossAgriVal,Y1988_GrossAgriVal,Y1989_GrossAgriVal,Y1990_GrossAgriVal,Y1991_GrossAgriVal,Y1992_GrossAgriVal,Y1993_GrossAgriVal,Y1994_GrossAgriVal,Y1995_GrossAgriVal,Y1996_GrossAgriVal,Y1997_GrossAgriVal,Y1998_GrossAgriVal,Y1999_GrossAgriVal,Y2000_GrossAgriVal,Y2001_GrossAgriVal,Y2002_GrossAgriVal,Y2003_GrossAgriVal,Y2004_GrossAgriVal,Y2005_GrossAgriVal,Y2006_GrossAgriVal,Y2007_GrossAgriVal,Y2008_GrossAgriVal,Y2009_GrossAgriVal,Y2010_GrossAgriVal,Y2011_GrossAgriVal,Y2012_GrossAgriVal,Y2013_GrossAgriVal,Y2014_GrossAgriVal,Y2015_GrossAgriVal,Y2016_GrossAgriVal,Y2017_GrossAgriVal,Y2018_GrossAgriVal,Y2019_GrossAgriVal,Y2020_GrossAgriVal,units_GrossAgriVal,Y1990_Pesticide,Y1991_Pesticide,Y1992_Pesticide,Y1993_Pesticide,Y1994_Pesticide,Y1995_Pesticide,Y1996_Pesticide,Y1997_Pesticide,Y1998_Pesticide,Y1999_Pesticide,Y2000_Pesticide,Y2001_Pesticide,Y2002_Pesticide,Y2003_Pesticide,Y2004_Pesticide,Y2005_Pesticide,Y2006_Pesticide,Y2007_Pesticide,Y2008_Pesticide,Y2009_Pesticide,Y2010_Pesticide,Y2011_Pesticide,Y2012_Pesticide,Y2013_Pesticide,Y2014_Pesticide,Y2015_Pesticide,Y2016_Pesticide,Y2017_Pesticide,Y2018_Pesticide,Y2019_Pesticide,Y2020_Pesticide,units_Pesticide,Y1990_Fire,Y1991_Fire,Y1992_Fire,Y1993_Fire,Y1994_Fire,Y1995_Fire,Y1996_Fire,Y1997_Fire,Y1998_Fire,Y1999_Fire,Y2000_Fire,Y2001_Fire,Y2002_Fire,Y2003_Fire,Y2004_Fire,Y2005_Fire,Y2006_Fire,Y2007_Fire,Y2008_Fire,Y2009_Fire,Y2010_Fire,Y2011_Fire,Y2012_Fire,Y2013_Fire,Y2014_Fire,Y2015_Fire,Y2016_Fire,Y2017

___
## 3. Generate Challenge Dataset

#### Break into Challenges
1. Challenge 1 
    - Try to predict next year's production (in tonnes) for crop (item) X, given last 20 years of data.
2. Challenge 2
    - Try to predict next year's production (in tonnes) for crop (item) X, given last 5 year of data.
2. Challenge 3
    - Try to predict next year's production (in tonnes) for crop (item) X, given last 1 year of data.

#### Challenge 1

In [340]:
df.columns[:20]

Index(['Area', 'Item Code', 'Item Code (CPC)', 'Item', 'Element Code', 'Element', 'Unit', 'Y1961_AgriProd_AgriProd', 'Y1962_AgriProd_AgriProd', 'Y1963_AgriProd_AgriProd', 'Y1964_AgriProd_AgriProd', 'Y1965_AgriProd_AgriProd', 'Y1966_AgriProd_AgriProd', 'Y1967_AgriProd_AgriProd', 'Y1968_AgriProd_AgriProd', 'Y1969_AgriProd_AgriProd', 'Y1970_AgriProd_AgriProd', 'Y1971_AgriProd_AgriProd', 'Y1972_AgriProd_AgriProd', 'Y1973_AgriProd_AgriProd'], dtype='object')

**Note**: Min yearis 1961, max year is 2020 on yield, let's base it off of this

In [66]:
keys = ['_AgriProd_AgriProd',
        '_AnnualFeedProd',
        '_AnnualFoodProd',
        '_AnnualAgriLosses',
        '_GDP',
        '_GrossAgriVal',
        '_Pesticide']

#NEWS DATA NOT INCLUDED HERE!

In [67]:
n = 20
combs_ = np.arange(1961,2021,1)
comb20 = []

for i in range(n,len(combs_)):
    comb20.append(combs_[i-n:i])

In [69]:
#Assemble Challenge 1 dataset
c1_dt = defaultdict(dict)
ct = 0

for idx,(k,v) in enumerate(df.iterrows()):
    
    if idx % 100 == 0:
        print("Done {}/{}...".format(idx,len(df)))
    
    area = v.Area
    areaCode = k
    item = v.Item
    prod_unit = v.Unit
    
    for j in comb20:
        
        c1_dt[ct] = {'area':area,
                      'areaCode':areaCode,
                      'item':item,
                      'prod_unit':prod_unit}
        
        for ijx,i in enumerate(j):

            for y in keys:
                
                try:
                    name = "Y{}{}".format(i,y)
                    val = v[name]
                    feat_name = 'y{}_{}'.format(ijx,y)
                    
                    c1_dt[ct][feat_name] = val
                    
                    
                except:
                    feat_name = 'y{}_{}'.format(ijx,y)
                    c1_dt[ct][feat_name] = np.nan
#                     print('Error!')
#                     print(name in v)

        ct+= 1

Done 0/2005...
Done 100/2005...
Done 200/2005...
Done 300/2005...
Done 400/2005...
Done 500/2005...
Done 600/2005...
Done 700/2005...
Done 800/2005...
Done 900/2005...
Done 1000/2005...
Done 1100/2005...
Done 1200/2005...
Done 1300/2005...
Done 1400/2005...
Done 1500/2005...
Done 1600/2005...
Done 1700/2005...
Done 1800/2005...
Done 1900/2005...
Done 2000/2005...


In [504]:
c1_df = pd.DataFrame.from_dict(c1_dt,orient='index')

In [527]:
c1_df.dropna(how='all',subset=c1_df.columns[4:],inplace=True) #drop all NA cols

In [528]:
c1_df.shape

(80030, 144)

In [529]:
c1_df.iloc[5000:5005]

,area,areaCode,item,prod_unit,y0__AgriProd_AgriProd,y0__AnnualFeedProd,y0__AnnualFoodProd,y0__AnnualAgriLosses,y0__GDP,y0__GrossAgriVal,y0__Pesticide,y1__AgriProd_AgriProd,y1__AnnualFeedProd,y1__AnnualFoodProd,y1__AnnualAgriLosses,y1__GDP,y1__GrossAgriVal,y1__Pesticide,y2__AgriProd_AgriProd,y2__AnnualFeedProd,y2__AnnualFoodProd,y2__AnnualAgriLosses,y2__GDP,y2__GrossAgriVal,y2__Pesticide,y3__AgriProd_AgriProd,y3__AnnualFeedProd,y3__AnnualFoodProd,y3__AnnualAgriLosses,y3__GDP,y3__GrossAgriVal,y3__Pesticide,y4__AgriProd_AgriProd,y4__AnnualFeedProd,y4__AnnualFoodProd,y4__AnnualAgriLosses,y4__GDP,y4__GrossAgriVal,y4__Pesticide,y5__AgriProd_AgriProd,y5__AnnualFeedProd,y5__AnnualFoodProd,y5__AnnualAgriLosses,y5__GDP,y5__GrossAgriVal,y5__Pesticide,y6__AgriProd_AgriProd,y6__AnnualFeedProd,y6__AnnualFoodProd,y6__AnnualAgriLosses,y6__GDP,y6__GrossAgriVal,y6__Pesticide,y7__AgriProd_AgriProd,y7__AnnualFeedProd,y7__AnnualFoodProd,y7__AnnualAgriLosses,y7__GDP,y7__GrossAgriVal,y7__Pesticide,y8__AgriProd_AgriProd,y8__AnnualFeedProd,y8__AnnualFoodProd,y8__AnnualAgriLosses,y8__GDP,y8__GrossAgriVal,y8__Pesticide,y9__AgriProd_AgriProd,y9__AnnualFeedProd,y9__AnnualFoodProd,y9__AnnualAgriLosses,y9__GDP,y9__GrossAgriVal,y9__Pesticide,y10__AgriProd_AgriProd,y10__AnnualFeedProd,y10__AnnualFoodProd,y10__AnnualAgriLosses,y10__GDP,y10__GrossAgriVal,y10__Pesticide,y11__AgriProd_AgriProd,y11__AnnualFeedProd,y11__AnnualFoodProd,y11__AnnualAgriLosses,y11__GDP,y11__GrossAgriVal,y11__Pesticide,y12__AgriProd_AgriProd,y12__AnnualFeedProd,y12__AnnualFoodProd,y12__AnnualAgriLosses,y12__GDP,y12__GrossAgriVal,y12__Pesticide,y13__AgriProd_AgriProd,y13__AnnualFeedProd,y13__AnnualFoodProd,y13__AnnualAgriLosses,y13__GDP,y13__GrossAgriVal,y13__Pesticide,y14__AgriProd_AgriProd,y14__AnnualFeedProd,y14__AnnualFoodProd,y14__AnnualAgriLosses,y14__GDP,y14__GrossAgriVal,y14__Pesticide,y15__AgriProd_AgriProd,y15__AnnualFeedProd,y15__AnnualFoodProd,y15__AnnualAgriLosses,y15__GDP,y15__GrossAgriVal,y15__Pesticide,y16__AgriProd_AgriProd,y16__AnnualFeedProd,y16__AnnualFoodProd,y16__AnnualAgriLosses,y16__GDP,y16__GrossAgriVal,y16__Pesticide,y17__AgriProd_AgriProd,y17__AnnualFeedProd,y17__AnnualFoodProd,y17__AnnualAgriLosses,y17__GDP,y17__GrossAgriVal,y17__Pesticide,y18__AgriProd_AgriProd,y18__AnnualFeedProd,y18__AnnualFoodProd,y18__AnnualAgriLosses,y18__GDP,y18__GrossAgriVal,y18__Pesticide,y19__AgriProd_AgriProd,y19__AnnualFeedProd,y19__AnnualFoodProd,y19__AnnualAgriLosses,y19__GDP,y19__GrossAgriVal,y19__Pesticide
5000,Bolivia (Plurinational State of),19,Rice,tonnes,34320.0,NaN,NaN,NaN,NaN,NaN,NaN,38571.0,NaN,NaN,NaN,NaN,NaN,NaN,42857.0,NaN,NaN,NaN,NaN,NaN,NaN,47142.0,NaN,NaN,NaN,NaN,NaN,NaN,51286.0,NaN,NaN,NaN,NaN,NaN,NaN,51714.0,NaN,NaN,NaN,NaN,NaN,NaN,56651.0,NaN,NaN,NaN,NaN,NaN,NaN,73560.0,NaN,NaN,NaN,NaN,NaN,NaN,83375.0,NaN,NaN,NaN,NaN,NaN,NaN,72530.0,NaN,NaN,NaN,1010.101010,NaN,NaN,85335.0,NaN,NaN,NaN,1094.276094,NaN,NaN,85200.0,NaN,NaN,NaN,1278.676194,NaN,NaN,77800.0,NaN,NaN,NaN,1299.350325,NaN,NaN,85200.0,NaN,NaN,NaN,2148.925537,NaN,NaN,126560.0,NaN,NaN,NaN,2398.800600,NaN,NaN,113045.0,NaN,NaN,NaN,2748.625687,NaN,NaN,121335.0,NaN,NaN,NaN,2762.917257,NaN,NaN,89315.0,NaN,NaN,NaN,3017.959618,NaN,NaN,76125.0,NaN,NaN,NaN,3272.704506,NaN,NaN,95225.0,NaN,NaN,NaN,3519.380952,NaN,NaN
5001,Bolivia (Plurinational State of),19,Rice,tonnes,38571.0,NaN,NaN,NaN,NaN,NaN,NaN,42857.0,NaN,NaN,NaN,NaN,NaN,NaN,47142.0,NaN,NaN,NaN,NaN,NaN,NaN,51286.0,NaN,NaN,NaN,NaN,NaN,NaN,51714.0,NaN,NaN,NaN,NaN,NaN,NaN,56651.0,NaN,NaN,NaN,NaN,NaN,NaN,73560.0,NaN,NaN,NaN,NaN,NaN,NaN,83375.0,NaN,NaN,NaN,NaN,NaN,NaN,72530.0,NaN,NaN,NaN,1010.101010,NaN,NaN,85335.0,NaN,NaN,NaN,1094.276094,NaN,NaN,85200.0,NaN,NaN,NaN,1278.676194,NaN,NaN,77800.0,NaN,NaN,NaN,1299.350325,NaN,NaN,85200.0,NaN,NaN,NaN,2148.925537,NaN,NaN,126560.0,NaN,NaN,NaN,2398.800600,NaN,NaN,113045.0,NaN,NaN,NaN,2748.625687,NaN,NaN,121335.0,NaN,NaN,NaN,2762.917257,NaN,NaN,89315.0,NaN,NaN,NaN,3017.959618,NaN,NaN,76125.0,NaN,NaN,NaN,3272.704506,NaN,NaN,9522

#### Challenge 2

In [509]:
n = 5
comb5 = []

for i in range(n,len(combs_)):
    comb5.append(combs_[i-n:i])

In [511]:
#Assemble Challenge 1 dataset
c2_dt = defaultdict(dict)
ct = 0

for idx,(k,v) in enumerate(df.iterrows()):
    
    if idx % 100 == 0:
        print("Done {}/{}...".format(idx,len(df)))
    
    area = v.Area
    areaCode = k
    item = v.Item
    prod_unit = v.Unit
    
    for j in comb5:
        
        c2_dt[ct] = {'area':area,
                      'areaCode':areaCode,
                      'item':item,
                      'prod_unit':prod_unit}
        
        for ijx,i in enumerate(j):

            for y in keys:
                
                try:
                    name = "Y{}{}".format(i,y)
                    val = v[name]
                    feat_name = 'y{}_{}'.format(ijx,y)
                    
                    c2_dt[ct][feat_name] = val
                    
                    
                except:
                    feat_name = 'y{}_{}'.format(ijx,y)
                    c2_dt[ct][feat_name] = np.nan

        ct+= 1

Done 0/2005...
Done 100/2005...
Done 200/2005...
Done 300/2005...
Done 400/2005...
Done 500/2005...
Done 600/2005...
Done 700/2005...
Done 800/2005...
Done 900/2005...
Done 1000/2005...
Done 1100/2005...
Done 1200/2005...
Done 1300/2005...
Done 1400/2005...
Done 1500/2005...
Done 1600/2005...
Done 1700/2005...
Done 1800/2005...
Done 1900/2005...
Done 2000/2005...


In [512]:
c2_df = pd.DataFrame.from_dict(c2_dt,orient='index')

In [526]:
c2_df.dropna(how='all',subset=c2_df.columns[4:],inplace=True) #drop all NA cols

(107162, 39)

In [513]:
c2_df.shape

(110275, 39)

In [514]:
c2_df.head(5)

,area,areaCode,item,prod_unit,y0__AgriProd_AgriProd,y0__AnnualFeedProd,y0__AnnualFoodProd,y0__AnnualAgriLosses,y0__GDP,y0__GrossAgriVal,y0__Pesticide,y1__AgriProd_AgriProd,y1__AnnualFeedProd,y1__AnnualFoodProd,y1__AnnualAgriLosses,y1__GDP,y1__GrossAgriVal,y1__Pesticide,y2__AgriProd_AgriProd,y2__AnnualFeedProd,y2__AnnualFoodProd,y2__AnnualAgriLosses,y2__GDP,y2__GrossAgriVal,y2__Pesticide,y3__AgriProd_AgriProd,y3__AnnualFeedProd,y3__AnnualFoodProd,y3__AnnualAgriLosses,y3__GDP,y3__GrossAgriVal,y3__Pesticide,y4__AgriProd_AgriProd,y4__AnnualFeedProd,y4__AnnualFoodProd,y4__AnnualAgriLosses,y4__GDP,y4__GrossAgriVal,y4__Pesticide
0,Armenia,1,Barley,tonnes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Armenia,1,Barley,tonnes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Armenia,1,Barley,tonnes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Armenia,1,Barley,tonnes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Armenia,1,Barley,tonnes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Challenge 3

In [516]:
n = 1
comb1 = []

for i in range(n,len(combs_)):
    comb1.append(combs_[i-n:i])

In [517]:
#Assemble Challenge 1 dataset
c3_dt = defaultdict(dict)
ct = 0

for idx,(k,v) in enumerate(df.iterrows()):
    
    if idx % 100 == 0:
        print("Done {}/{}...".format(idx,len(df)))
    
    area = v.Area
    areaCode = k
    item = v.Item
    prod_unit = v.Unit
    
    for j in comb1:
        
        c3_dt[ct] = {'area':area,
                      'areaCode':areaCode,
                      'item':item,
                      'prod_unit':prod_unit}
        
        for ijx,i in enumerate(j):

            for y in keys:
                
                try:
                    name = "Y{}{}".format(i,y)
                    val = v[name]
                    feat_name = 'y{}_{}'.format(ijx,y)
                    
                    c3_dt[ct][feat_name] = val
                    
                    
                except:
                    feat_name = 'y{}_{}'.format(ijx,y)
                    c3_dt[ct][feat_name] = np.nan

        ct+= 1

Done 0/2005...
Done 100/2005...
Done 200/2005...
Done 300/2005...
Done 400/2005...
Done 500/2005...
Done 600/2005...
Done 700/2005...
Done 800/2005...
Done 900/2005...
Done 1000/2005...
Done 1100/2005...
Done 1200/2005...
Done 1300/2005...
Done 1400/2005...
Done 1500/2005...
Done 1600/2005...
Done 1700/2005...
Done 1800/2005...
Done 1900/2005...
Done 2000/2005...


In [518]:
c3_df = pd.DataFrame.from_dict(c3_dt,orient='index')

In [530]:
c3_df.dropna(how='all',subset=c3_df.columns[4:],inplace=True) #drop all NA cols

In [531]:
c3_df.shape

(112917, 11)

In [532]:
c3_df.head(5)

,area,areaCode,item,prod_unit,y0__AgriProd_AgriProd,y0__AnnualFeedProd,y0__AnnualFoodProd,y0__AnnualAgriLosses,y0__GDP,y0__GrossAgriVal,y0__Pesticide
9,Armenia,1,Barley,tonnes,NaN,NaN,NaN,NaN,0.0,NaN,NaN
10,Armenia,1,Barley,tonnes,NaN,NaN,NaN,NaN,0.0,NaN,NaN
11,Armenia,1,Barley,tonnes,NaN,NaN,NaN,NaN,0.0,NaN,NaN
12,Armenia,1,Barley,tonnes,NaN,NaN,NaN,NaN,0.0,NaN,NaN
13,Armenia,1,Barley,tonnes,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [535]:
#Save Out
c1_df.to_json('data/challenge1_dataset.json')
c2_df.to_json('data/challenge2_dataset.json')
c3_df.to_json('data/challenge3_dataset.json')

____
## End